## Example with a Single Qubit Varational Form

We will now use the simple single qubit variational form to solve a problem
similar to ground state energy estimation. Specifically, we are given
a random probability vector $\hat{x}$ and wish to determine a possible parameterization for our single qubit variational form such that it outputs a probability distribution that is close to $\hat{x}$ (where closeness is defined in terms of the Manhattan distance between the two probability vectors).

In [1]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.aqua.components.optimizers import COBYLA

Create the random probability vector

In [32]:
np.random.seed(999999)
target_distr = np.random.rand(2**2)  # 2 dimensional vector
# normalize the vector to get a valid probability vector
target_distr /= sum(target_distr)

print(target_distr)

[0.22027533 0.20863466 0.51316198 0.05792803]


A function that takes parameters of a single U3 variational form as arguments
as returns corresponding quantum circuit

In [33]:
def get_var_form(params):
    qr = QuantumRegister(2, name="q")
    cr = ClassicalRegister(2, name="c")
    qc = QuantumCircuit(qr, cr)
    
    #qc.u3(params[0], params[1], params[2], qr[0])
    qc.h(qr[0])
    qc.cx(qr[0], qr[1])
    qc.measure(qr, cr)
    
    return qc

In [34]:
print(get_var_form([np.pi, .5*np.pi, -np.pi]))

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 


An objective function that takes in as input a list of the variational form's parameters, and returns
the cost associated with those parameters

In [37]:
qasm_backend = Aer.get_backend('qasm_simulator')
NUM_SHOTS = 10000
params = [np.pi, np.pi/2, -np.pi]

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    
    if (len(output_distr) == 1):
        output_distr.append(1 - output_distr[0])
        
    return output_distr

def objective_function(params):
    # get a quantum circuit instance from the params
    qc = get_var_form(params)
    
    # execute quantum circuit to obtain the probability distribution associated with current parameters
    result = execute(qc, qasm_backend, shots=NUM_SHOTS).result()
    
    # obtain counts for each measured state and convert the counts into a probability vector
    counts = result.get_counts(qc)
    output_distr = get_probability_distribution(counts)
    
    print("counts:",counts)
    print("output distr:", output_distr)
    print("target distr:", target_distr)
    
    cost = sum( [np.abs(output_distr[i] -  target_distr[i]) for i in range(2)] )
    return cost

print(objective_function(params))

counts: {'00': 5084, '11': 4916}
output distr: [0.5084, 0.4916]
target distr: [0.22027533 0.20863466 0.51316198 0.05792803]
0.5710900130147567


Finally, we create an instance of the COBYLA optimizer, and run the algorithm. Note that the output varies from run to run. Moreover, while close, the obtained distribution might not be exactly the same as the target distribution, however, increasing the number of shots taken will increase the accuracy of the output.

In [43]:
# initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# create the initial parameters (noting that our single qubit variational formhas 3 parameters)
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

# obtain the output distribution using the final parameters
qc = get_var_form(ret[0])
counts = execute(qc, qasm_backend, shots=NUM_SHOTS).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])

{'00': 8977, '01': 1023} [0.8977, 0.1023] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 4604, '01': 5396} [0.4604, 0.5396] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 4595, '01': 5405} [0.4595, 0.5405] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 4613, '01': 5387} [0.4613, 0.5387] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 620, '01': 9380} [0.062, 0.938] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 2277, '01': 7723} [0.2277, 0.7723] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 4512, '01': 5488} [0.4512, 0.5488] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 4650, '01': 5350} [0.465, 0.535] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 4561, '01': 5439} [0.4561, 0.5439] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 4502, '01': 5498} [0.4502, 0.5498] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 4287, '01': 5713} [0.4287, 0.5713] [0.22027533 0.20863466 0.51316198 0.05792803]
{'00': 5200, '01': 4800} [0.52, 0.48]